In [1]:
# https://api.mongodb.com/python/current/tutorial.html
import pprint
from pymongo import MongoClient
import pandas as pd
import requests as req 
import json
import math
import re
import folium
from folium import plugins
client = MongoClient('mongodb://localhost:27017/')
db = client.ejercicio

In [2]:
# {$and:[{deadpooled_year:null}, {number_of_employees:{$ne:null}}, {"offices.latitude":{$ne:null}}, {"offices.latitude":{$exists:true}}]}

In [3]:
empresas= db.companhias.find({"$and":[{"deadpooled_year":None}, {"number_of_employees":{"$ne":None}},{"total_money_raised":{"$ne":None}}, {"offices.latitude":{"$ne":None}}, {"offices.latitude":{"$exists":"true"}}]})

In [4]:
df=pd.DataFrame(empresas)[["name","category_code","number_of_employees","founded_year", "total_money_raised","offices"]]

(4839, 6)

In [6]:
df.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digg,news,60,2004.0,$45M,"[{'description': None, 'address1': '135 Missis..."
1,Geni,web,18,2006.0,$16.5M,"[{'description': 'Headquarters', 'address1': '..."
2,Fox Interactive Media,web,0,1979.0,$0,"[{'description': '', 'address1': '407 N Maple ..."
3,Scribd,news,50,2007.0,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,15,2007.0,$5.58M,"[{'description': None, 'address1': '', 'addres..."


In [7]:
df1=df.copy()

In [8]:
res = req.get("https://api.exchangeratesapi.io/latest?base=USD").json()

In [9]:
def cambiar(string):
    if string.startswith("$"):
        string=string.replace("$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= float(string)*1000000
        elif string.endswith("k"):
            string=string.replace("k","")
            string= float(string)*1000
        elif string.endswith("B"):
            string=string.replace("B","")
            string= float(string)*1000000000
        else:
            string=float(string)
    
    elif string.startswith("€"):
        string=string.replace("€","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["EUR"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["EUR"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["EUR"]
        else:
            string=float(string)
            
    elif string.startswith("£"):
        string=string.replace("£","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["GBP"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["GBP"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["GBP"]
        else:
            string=float(string)

    elif string.startswith("¥"):
        string=string.replace("¥","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["JPY"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["JPY"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["JPY"]
        else:
            string=float(string)
      
    elif string.startswith("C$"):
        string=string.replace("C$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["CAD"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["CAD"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["CAD"]
        else:
            string=float(string)
            
    elif string.startswith("kr"):
        string=string.replace("kr","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["SEK"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["SEK"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["SEK"]
        else:
            string=float(string)
        
    return string

In [10]:
lst=[]
for a in range(len(df1)):
    lst.append(df1["offices"][a][0]["city"])
   

In [11]:
df1["ciudad"]=lst

In [13]:
df1.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices,ciudad
0,Digg,news,60,2004.0,$45M,"[{'description': None, 'address1': '135 Missis...",San Francisco
1,Geni,web,18,2006.0,$16.5M,"[{'description': 'Headquarters', 'address1': '...",West Hollywood
2,Fox Interactive Media,web,0,1979.0,$0,"[{'description': '', 'address1': '407 N Maple ...",Beverly Hills
3,Scribd,news,50,2007.0,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant...",San Francisco
4,MeetMoi,social,15,2007.0,$5.58M,"[{'description': None, 'address1': '', 'addres...",New York City


In [14]:
df1["total_money_raised"]=df1["total_money_raised"].apply(cambiar)

In [15]:
df1.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices,ciudad
0,Digg,news,60,2004.0,45000000.0,"[{'description': None, 'address1': '135 Missis...",San Francisco
1,Geni,web,18,2006.0,16500000.0,"[{'description': 'Headquarters', 'address1': '...",West Hollywood
2,Fox Interactive Media,web,0,1979.0,0.0,"[{'description': '', 'address1': '407 N Maple ...",Beverly Hills
3,Scribd,news,50,2007.0,25800000.0,"[{'description': 'HQ', 'address1': '539 Bryant...",San Francisco
4,MeetMoi,social,15,2007.0,5580000.0,"[{'description': None, 'address1': '', 'addres...",New York City


In [16]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df1[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [17]:
first_office.head()

,lat,lng,oficina_principal,totalOffices
0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1
1,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1
2,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ...",1
3,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",1
4,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",1


In [31]:
df_clean = pd.concat([df1,first_office], axis=1)[["name","lng","lat", "oficina_principal","totalOffices", "category_code", "number_of_employees", "founded_year", "total_money_raised", "ciudad"]]

In [32]:
df_clean.head()

,name,lng,lat,oficina_principal,totalOffices,category_code,number_of_employees,founded_year,total_money_raised,ciudad
0,Digg,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",1,news,60,2004.0,45000000.0,San Francisco
1,Geni,-118.393064,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",1,web,18,2006.0,16500000.0,West Hollywood
2,Fox Interactive Media,-118.394170,34.076179,"{'type': 'Point', 'coordinates': [-118.39417, ...",1,web,0,1979.0,0.0,Beverly Hills
3,Scribd,-122.404052,37.789634,"{'type': 'Point', 'coordinates': [-122.404052,...",1,news,50,2007.0,25800000.0,San Francisco
4,MeetMoi,-73.985506,40.757929,"{'type': 'Point', 'coordinates': [-73.985506, ...",1,social,15,2007.0,5580000.0,New York City


In [34]:
df_clean.to_json('limpio.json', orient="records")

In [35]:
df_clean.to_csv('limpio.csv')

In [36]:
oficinas=df_clean.copy()

In [39]:
oficinas=oficinas[(oficinas["total_money_raised"]>1000000) & (oficinas["founded_year"]>2005)]
lista_paises=list(oficinas.ciudad.value_counts().head(20).index)
print(lista_paises)

['San Francisco', 'New York', 'London', 'Seattle', 'Mountain View', 'San Mateo', 'Palo Alto', 'Boulder', 'Los Angeles', 'San Jose', 'Paris', 'Chicago', 'Boston', 'Portland', 'Berlin', 'Cambridge', 'Redwood City', 'Burlington', 'Madrid', 'Sunnyvale']


In [116]:
for c in lista_paises:
    i=0
    medias=[]
    bd = oficinas[oficinas["ciudad"]==c]
    contador=bd.ciudad.count()
    suma=bd.total_money_raised.sum()
    media=round(suma/contador)
    medias.append(c, media)
    i+=1
print(medias) 

TypeError: append() takes exactly one argument (2 given)

In [65]:
bd.ciudad

Series([], Name: ciudad, dtype: object)

In [ ]:
m = folium.Map([41.8781, -87.6298], zoom_start=2)
stationArr = oficinas[['lat', 'lng']].values
m.add_child(plugins.HeatMap(stationArr, radius=15))
m

In [ ]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html
def findNear(geopoint, radio_max_meters=2000):
    return db.empresas.find({"oficina_principal": {"$near": {"$geometry": geopoint,"$maxDistance": radio_max_meters,}}})


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))